In [4]:
import numpy as np
import plotly.graph_objects as go
import os
import pickle 
import pandas as pd

filepath = 'C:/Users/Michael Forsthofer/OneDrive - University of Sussex/BadenLab/2p-imaging/singleplane/St53/'
pkl_files = [f for f in os.listdir(filepath) if f.endswith('.pkl')]

stimuli = ['moving stripes', 'dots_ON', 'dots_OFF', 'loom']

depths = {}
for stim in stimuli:
    depths[stim] = []


fig = go.Figure()

for pkl_file in pkl_files:
    with open(filepath+pkl_file, 'rb') as f:
        extracted_data_loaded = pickle.load(f)
        print(extracted_data_loaded['metadata'])
        for stim in stimuli:
            flattened_list = extracted_data_loaded['depths'][stim]
            if len(flattened_list)>0:
                flattened_list = np.hstack(flattened_list)
            depths[stim].append(flattened_list)
            if stim == 'loom':
                fig.add_trace(go.Histogram(x=flattened_list))

fig.update_layout(barmode='overlay', width=600, height=400,
                  margin=dict(l=20, r=20, t=20, b=20),
                  xaxis_title='Relative OT Depth',
                  yaxis_title='# ROIs')           
fig.update_traces(opacity=0.3)   
fig.show()



{'SD_threshold': 1.2, 'correlation_threshold': 0.68, 'min_traces_per_cluster': 6, 'cluster_threshold': 45, 'depth': 250}
{'SD_threshold': 1.4, 'correlation_threshold': 0.34, 'min_traces_per_cluster': 6, 'cluster_threshold': 35, 'depth': 266}
{'SD_threshold': 1.4, 'correlation_threshold': 0.6, 'min_traces_per_cluster': 6, 'cluster_threshold': 35, 'depth': 245}


In [3]:
stripes = np.hstack(depths['moving stripes'])
loom = np.hstack(depths['loom'])
dots_on = np.hstack(depths['dots_ON'])
dots_off = np.hstack(depths['dots_OFF'])

flattened_list = np.hstack([stripes, loom, dots_on, dots_off])

fig = go.Figure()
fig.add_trace(go.Histogram(y=stripes, name='moving stripes'))
fig.add_trace(go.Histogram(y=loom, name='loom'))
fig.add_trace(go.Histogram(y=dots_on,name='dots_ON'))
fig.add_trace(go.Histogram(y=dots_off, name='dots_OFF'))
fig.update_layout(barmode='overlay', width=600, height=400,
                  margin=dict(l=20, r=20, t=20, b=20),
                  xaxis_title='# ROIs',
                  yaxis_title='relative depth')  
fig.update_traces(opacity=0.3)    
